In [10]:
from datetime import datetime, timedelta
import requests
import pandas as pd
import psycopg2

In [11]:
TMO_URL = 'https://api.louismmoo.com/api/viettinbank/transactions'
PAYMENT_TYPE_DLBT = "DLBT"
PAYMENT_TYPE_DLBT60 = "DLBT60"
BANK_ACCOUNT_AUTO_STATUS_NONE = 0 
BANK_ACCOUNT_STATUS_ACTIVE = 1

In [12]:
def extract_bank_acc():
      rawsql = f"""
            SELECT 
                  ba.login_name as  username,
                  ba.password,
                  ba.account_no,
                  ba.id  as bank_account_id ,
                  b.code as bank_code,
                  b.id as bank_id
            FROM bank_account AS ba 
            LEFT JOIN bank AS b ON b.id = ba.bank_id 
            WHERE auto_status  != '{BANK_ACCOUNT_AUTO_STATUS_NONE}' AND status = '{BANK_ACCOUNT_STATUS_ACTIVE}'
            """
            
      # bank_acc_df = conn_payment_pg_hook.get_pandas_df(rawsql)
            
      # Temp <<========================================
      with psycopg2.connect(
            database="paymentDB",
            host='172.17.0.1',
            user='airflow',
            password='airflow',
            port='5432'
      ) as conn:
            sql = rawsql
            bank_acc_df = pd.read_sql_query(sql, conn)
            
      conn.close()
      # ========================================>> Temp 

      return bank_acc_df

In [60]:
def fetch_online_bank_data(username,password,accountNumber,begin,end, page):
    try: 
        payload = {
            "username":username,
            "password":password ,
            "accountNumber":accountNumber,
            "begin":begin,
            "end":end,
            "page":page
        }
        
        req =requests.post(
            TMO_URL, 
            data = payload
        )

        result = req.json().get('data',{}).get('transactions', [])
        
        print(payload)
        print(result)
        
        df = pd.DataFrame.from_records(result)  
        return df
    
    except:
        print("Failed to Fetch Online Bank Data")
        exit(1)

def get_new_online_bank_df(begin,end, page):
    bank_acc_df = extract_bank_acc()
    
    result = pd.DataFrame()
    
    for index, row in bank_acc_df.iterrows():
        trans_df = fetch_online_bank_data(
            row['username'], 
            row['password'], 
            row['account_no'],
            begin, 
            end,
            page
        )
        
        if not trans_df.empty:
            df = trans_df.loc[:, ['trxId','remark','amount','processDate']]
            df = df.rename(columns={
                'trxId':'bank_reference',
                'remark':'bank_description',
                'amount':'net_amount',
                'processDate':'transaction_date'
            })
            df['bank_account_id'] = row['bank_account_id']
            df['bank_id'] = row['bank_id']
            result = pd.concat([result, df])

    return result

In [36]:
today = datetime.now().strftime("%d/%m/%Y")

In [61]:
df = get_new_online_bank_df(today, today, 0)

/tmp/ipykernel_58813/2011876010.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  bank_acc_df = pd.read_sql_query(sql, conn)


{'username': '0843501988', 'password': 'Err7412!', 'accountNumber': '103878535119', 'begin': '09/09/2023', 'end': '09/09/2023', 'page': 0}
[{'currency': 'VND', 'remark': 'CT DI:904K2390DRW9QCNU CK', 'amount': '29800000.00', 'balance': '189212.00', 'trxId': '904K2390DRW9QCNU', 'processDate': '09-09-2023 15:46:39', 'dorC': 'D', 'refType': 'HistoryRecordSequence', 'refId': '2727', 'tellerId': 'ZES', 'corresponsiveAccount': '1032953065', 'corresponsiveName': 'NGUYEN TRONG DU', 'channel': '78 - Retail Internet Banking', 'serviceBranchId': '90432', 'serviceBranchName': 'CN 2 - TP HCM - PGD PHAN XICH LONG', 'pmtType': 'RTGS - Chuyển tiền nhanh', 'sendingBankId': '01201001', 'sendingBranchId': '', 'sendingBranchName': '', 'receivingBankId': 'BFTVVNVX', 'receivingBranchId': '', 'receivingBranchName': 'VCB_NH TMCP Ngoai thuong VN'}, {'currency': 'VND', 'remark': '927256', 'amount': '2000000.00', 'balance': '29989212.00', 'trxId': '14Xzm-7l6BCR74L', 'processDate': '09-09-2023 15:21:28', 'dorC': '

In [62]:
df.head()

,bank_reference,bank_description,net_amount,transaction_date,bank_account_id,bank_id
0,904K2390DRW9QCNU,CT DI:904K2390DRW9QCNU CK,29800000.00,09-09-2023 15:46:39,1,1
1,14Xzm-7l6BCR74L,927256,2000000.00,09-09-2023 15:21:28,1,1
2,1YnPQ-7l6Ag1FdV,693585,7000000.00,09-09-2023 15:13:29,1,1
3,904S2390DQEVNBMV,CT DEN:325208361892 198895,300000.00,09-09-2023 15:09:14,1,1
4,904S2390DQ2CGD5W,CT DEN:325207832308 414592,4900000.00,09-09-2023 14:59:46,1,1
